In [1]:
import numpy as np
import nibabel as nib
import pickle
import time
import concurrent.futures
import torch
from sklearn.preprocessing import normalize
from scipy.stats import pearsonr
import os
import pandas as pd

In [2]:
imp_roi = np.array([10,11,12,13,17,18,49,50,51,52,53,54,
                11101, 11102, 11104, 11105, 11106, 11107, 11108, 11109, 11111, 11112, 11113, 11114, 11115, 11116, 11117, 11118, 11119, 11120,
                11121, 11122, 11123, 11124, 11125, 11126, 11127, 11128, 11129, 11130, 11133, 11134, 11135, 11136, 11137, 11138, 11141, 11143,
                11145, 11146, 11147, 11148, 11149, 11150, 11153, 11154, 11155, 11157, 11158, 11159, 11160, 11161, 11162, 11165, 11166,
                11167, 11168, 11169, 11170, 11172, 11173, 11174, 
                12101, 12102, 12104, 12105, 12106, 12107, 12108, 12109, 12111, 12112, 12113, 12114, 12115, 12116, 12117, 12118, 12119, 12120,
                12121, 12122, 12123, 12124, 12125, 12126, 12127, 12128, 12129, 12130, 12133, 12134, 12135, 12136, 12137, 12138, 12141, 12143, 
                12145, 12146, 12147, 12148, 12149, 12150, 12153, 12154, 12155, 12157, 12158, 12159, 12160, 12161, 12162, 12165, 12166,
                12167, 12168, 12169, 12170, 12172, 12173, 12174])

enable_prallel_processing = 0 # 0-disable 1-enable
no_sub_to_run_in_parallel = 4
sub_det_df = pd.read_excel('data/rs_fmri_sub_details_WO_AS.xlsx')
cur_sub_list_before = sub_det_df['Subject_ID_before'].to_numpy()
cur_sub_list_after = sub_det_df['Subject_ID_after'].to_numpy()
subject_ids = np.concatenate((cur_sub_list_before,cur_sub_list_after))
print('# of subjects to Process:', len(subject_ids))
data_dir = '/data-braingut/bharath/fmri_bandpass'
data_dir1 = '/data-braingut/bharath/fmri_bandpass'

In [7]:
def extract_hist_FC(cur_sub,imp_roi,no_bins,data_dir,data_dir1):
    brik1 = nib.load(('%s/%s/errts.%s.fanaticor+orig.BRIK')%(data_dir,cur_sub,cur_sub))
    fmri_img1 = brik1.get_fdata()
    file_path_seg = ('%s/%s/follow_ROI_aeseg+orig.BRIK.gz')%(data_dir1,cur_sub)
    brik_seg = nib.load(file_path_seg)    
    seg_img = np.squeeze(brik_seg.get_fdata())
    
    mask_brik = nib.load('%s/%s/full_mask.%s+orig.BRIK.gz'%(data_dir1,cur_sub,cur_sub))
    mask_img = np.squeeze(mask_brik.get_fdata())
    seg_img[mask_img == 0] = 0      
    
# put all the BOLD signals from the selected ROI's in a matric and note the each ROI locatopn in the matrix 
    all_bols_mask = np.empty((fmri_img1.shape[3],0))
    Roi_tuplets = {}  
    count = 0
    for idx in imp_roi:
        roi_bold = fmri_img1[seg_img == idx,:].transpose()
        all_bols_mask = np.hstack((all_bols_mask,roi_bold))
        Roi_tuplets[idx] = (count,roi_bold.shape[1])
        count = count + roi_bold.shape[1]
    
# pearson correlation is calculated by first normalizing all the bold signals and then taking the inner product     

    all_bols_mask = normalize(all_bols_mask, axis = 0) # normalize the bold matrix
    
# calculate all inner products at onece using matric multiplication (using torch matrix multiplication as iit is faster than numpy matrix multiplication)  
    data = torch.from_numpy(all_bols_mask)
    corr_mat_gpu = torch.mm(torch.transpose(data, 0, 1),data)
    corr_mat_gpu = corr_mat_gpu.arctanh()
    corr_mat_gpu_cpu = corr_mat_gpu.data.numpy()
    
#    claculate the histogran of corelation values between all pairs of ROI's
    hist_dict = {}
    for i,roi1 in enumerate(imp_roi):
        for j,roi2 in enumerate(imp_roi[i:]):
            i1 = Roi_tuplets[roi1][0]
            j1 = Roi_tuplets[roi2][0]
            if(i1>j1):
                temp_mat = corr_mat_gpu_cpu[i1:i1+Roi_tuplets[roi1][1],j1:j1+Roi_tuplets[roi2][1]]
            else:
                temp_mat = corr_mat_gpu_cpu[j1:j1+Roi_tuplets[roi2][1],i1:i1+Roi_tuplets[roi1][1]]

            hist_val = np.histogram(temp_mat, bins = no_bins, range = (-1,1), density=True)
            hist_dict[roi1,roi2] = hist_val[0]
    print(cur_sub)    
    return(hist_dict)

In [1]:
no_bins = 50
start = time.time()
all_sub_hist = {}
if(enable_prallel_processing == 1):
    with concurrent.futures.ProcessPoolExecutor(max_workers = no_sub_to_run_in_parallel) as executor:
        corr_handle = {executor.submit(extract_conn_hist_GPU, sub_id, imp_roi,no_bins,data_dir,data_dir1): sub_id for sub_id in subject_ids}
        for id_sub in concurrent.futures.as_completed(corr_handle):
            all_sub_hist[corr_handle[id_sub]] = id_sub.result()
else:
    for sub_id in subject_ids:
        all_sub_hist[sub_id] = extract_conn_hist_GPU(sub_id,imp_roi,no_bins,data_dir,data_dir1)
end = time.time()
print(end - start)
print(len(all_sub_hist))

with open('data/FC_Hist_avg/fmri_paper_subjects_141ROI_hist_A2009_bandpass_fisher_z_2890.pickle', 'wb') as handle:
    pickle.dump(all_sub_hist, handle, protocol=pickle.HIGHEST_PROTOCOL)